<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'>Info</font></h3>
    
In this notebook the model for the Recommender System is created.

In [50]:
import time
import mlflow
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

from typing import Dict, Text

# MLFlow setup

In [51]:
experiment_name = "Joropo_Expreriment"

if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(name=experiment_name) 

experiment = mlflow.get_experiment_by_name(experiment_name)

# Variables

In [52]:
path_data = "../data/processed/Final_data_for_ML.parquet"
path_model = '../models'
epochs = 1000
embedding_dimension = 64
learning_rate = 0.5

# Function definition

In [53]:
def find_lowest_value(array):
  lowest_value = array[0]
  for value in array:
    if value < lowest_value:
      lowest_value = value
  return lowest_value

# Load data

In [54]:
df = pd.read_parquet(path_data)
df = df[:1000]
total_ratings= len(df.index)

In [55]:
df.sample(1)

,title,reviewerID,reviewTime
372,The Rheumatoid Arthritis Cookbook: Anti-Inflam...,A132WCFSP8JCYI,1512604800000000


# Prepare data

In [ ]:
ratings = tf.data.Dataset.from_tensor_slices( {"reviewerID":tf.cast( df.reviewerID.values, tf.string),
                                                "reviewTime":tf.cast( df.reviewTime.values, tf.string),
                                                "title" : tf.cast( df.title.values, tf.string)} )                                           

In [ ]:
ratings = ratings.map(lambda x: {
    "title": x["title"],
    "reviewerID": x["reviewerID"],
    "reviewTime": x["reviewTime"]
})

In [ ]:
products = ratings.map(lambda x: x["title"])

In [ ]:
products

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
train = shuffled.take( int(total_ratings*0.8) )
test = shuffled.skip(int(total_ratings*0.8)).take(int(total_ratings*0.2))

In [ ]:
product_titles = products.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["reviewerID"])

unique_product_titles = np.unique(np.concatenate(list(product_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# Create the model

In [ ]:
product_title_lookup = tf.keras.layers.StringLookup()
product_title_lookup.adapt(ratings.map(lambda x: x["title"]))
product_title_embedding = tf.keras.layers.Embedding(input_dim=product_title_lookup.vocabulary_size(),output_dim=64)
product_model = tf.keras.Sequential([product_title_lookup, product_title_embedding])

In [ ]:
user_id_lookup = tf.keras.layers.StringLookup()
user_id_lookup.adapt(ratings.map(lambda x: x["reviewerID"]))
user_id_embedding = tf.keras.layers.Embedding(user_id_lookup.vocabulary_size(), 64)
user_model = tf.keras.Sequential([user_id_lookup, user_id_embedding])

In [ ]:
timestamp_normalization = tf.keras.layers.Normalization(axis=None)
timestamp_normalization.adapt(ratings.map(lambda x: x["reviewTime"]).batch(1024))


In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=products.batch(128).map(user_model)
)

In [ ]:
task = tfrs.tasks.Retrieval(
  #metrics=metrics
  metrics=[]
)

In [ ]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.product_model: tf.keras.Model = product_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features["reviewerID"])
    positive_movie_embeddings = self.product_model(features["title"])

    return self.task(user_embeddings, positive_movie_embeddings)

In [ ]:
model = MovielensModel(user_model, product_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=learning_rate))

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
Loss_call_back = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3, verbose=1, mode='min')

# Fit the model

In [ ]:
model_hist = model.fit(cached_train, 
                        epochs=epochs, 
                        callbacks=[Loss_call_back])

# Evaluate the model

In [ ]:
evaluate_history = model.evaluate(cached_test, 
                return_dict=True)

# Log params and metrics in MLFlow

In [ ]:
# mlflow.start_run(experiment_id=experiment.experiment_id)

# mlflow.log_param("epochs", epochs)
# mlflow.log_param("embedding_dimension", embedding_dimension)
# mlflow.log_param("learning_rate", learning_rate)

# mlflow.log_metric("Loss", find_lowest_value(model_hist.history['loss']))
# mlflow.log_metric("y_Loss", evaluate_history["loss"])

# mlflow.end_run()

# Manual prediction

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

index.index_from_dataset(tf.data.Dataset.zip((products.batch(100), products.batch(100).map(model.product_model))))

_, titles = index(tf.constant(["42"]))

print(f"Recommendations for user 42: {titles[0, :3]}")

# Save model for deployment

In [ ]:
tf.saved_model.save(index, path_model)
loaded = tf.saved_model.load(path_model)

In [ ]:
scores, titles = loaded(["A245831QTLS0K3"])
print(f"Recommendations: {titles[0][:3]}")

In [ ]:
print(f"Recommendations: {scores[0][:3]}")